In [5]:
import pandas as pd
from pathlib import Path

#authors:  Acarie Jenkens, Carrie Bates, Corey Monsma, Parker Teske, Stargliy Bobby

# File to Load (use the absolute path per class exercise. Used on an Windows maching)
opioids_path = Path("Resources/Accidental_Drug_Related_Deaths_2012-2022.csv")

# Read opioid csv file from data.gov then store into Pandas DataFrames
opioids_data_df = pd.read_csv(opioids_path)

#needed to rename column names for easier import into SQL database columns
opioids_data_df = opioids_data_df.rename(columns={"Date Type":"DateType","Residence City":"ResidenceCity",
                                                  "Residence County":"ResidenceCounty","Residence State":"ResidenceState",
                                                  "Injury City":"InjuryCity","Injury County":"InjuryCounty",
                                                  "Injury State":"InjuryState","Injury Place":"InjuryPlace",
                                                  "Death City":"DeathCity","Death County":"DeathCounty",
                                                  "Description of Injury":"DescriptionofInjury",
                                                  "Death State":"DeathState","Location if Other":"LocationifOther",
                                                  "Cause of Death":"CauseofDeath","Manner of Death":"MannerofDeath",
                                                  "Other Significant Conditions ":"OtherSignificantConditions",
                                                  "Heroin death certificate (DC)":"Heroindeathcertificate(DC)",
                                                  "Fentanyl Analogue":"FentanylAnalogue","Morphine (Not Heroin)":"Morphine(NotHeroin)",
                                                  "Opiate NOS":"OpiateNOS","Other Opioid":"OtherOpioid",
                                                  "Any Opioid":"AnyOpioid"})
opioids_data_df.head()

,Date,DateType,Age,Sex,Race,Ethnicity,ResidenceCity,ResidenceCounty,ResidenceState,InjuryCity,...,Xylazine,Gabapentin,OpiateNOS,Heroin/Morph/Codeine,OtherOpioid,AnyOpioid,Other,ResidenceCityGeo,InjuryCityGeo,DeathCityGeo
0,5/29/2012,Date of death,37.0,Male,Black,NaN,STAMFORD,FAIRFIELD,NaN,STAMFORD,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"STAMFORD, CT\n(41.051924, -73.539475)","STAMFORD, CT\n(41.051924, -73.539475)","CT\n(41.575155, -72.738288)"
1,6/27/2012,Date of death,37.0,Male,White,NaN,NORWICH,NEW LONDON,NaN,NORWICH,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"NORWICH, CT\n(41.524304, -72.075821)","NORWICH, CT\n(41.524304, -72.075821)","Norwich, CT\n(41.524304, -72.075821)"
2,3/24/2014,Date of death,28.0,Male,White,NaN,HEBRON,NaN,NaN,HEBRON,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"HEBRON, CT\n(41.658069, -72.366324)","HEBRON, CT\n(41.658069, -72.366324)","Marlborough, CT\n(41.632043, -72.461309)"
3,12/31/2014,Date of death,26.0,Female,White,NaN,BALTIC,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"BALTIC, CT\n(41.617221, -72.085031)","CT\n(41.575155, -72.738288)","Baltic, CT\n(41.617221, -72.085031)"
4,1/16/2016,Date of death,41.0,Male,White,NaN,SHELTON,FAIRFIELD,CT,SHELTON,...,NaN,NaN,NaN,NaN,NaN,Y,NaN,"SHELTON, CT\n(41.316843, -73.092968)","SHELTON, CT\n(41.316843, -73.092968)","Bridgeport, CT\n(41.179195, -73.189476)"


In [6]:
#next clean the NaN within the overdose cause columns
#next clean the NaN within the overdose cause columns
clean_opioid_df = opioids_data_df

clean_opioid_df.fillna("N", inplace=True)

clean_opioid_df.drop(clean_opioid_df[clean_opioid_df['Sex'] == 'N'].index,inplace=True)
clean_opioid_df.drop(clean_opioid_df[clean_opioid_df['Age'] == 'N'].index,inplace=True)

#drop the two columns ResidenceCityGeo and InjuryCityGeo duplicating and inconsistent
clean_opioid_df = clean_opioid_df.drop('ResidenceCityGeo', axis=1)
clean_opioid_df = clean_opioid_df.drop('InjuryCityGeo', axis=1)

#need to split the DeathCityGeo to pull out the lat and longs
#pull the DeathCityGeo into a series
deathCity_srs = clean_opioid_df['DeathCityGeo']
#parse the carrige returns \n into a new dataframe
deathCity_geo_df = deathCity_srs.str.split('\n',expand=True)
#pull the lats and longs into another series
#df['range'].str.replace(',','-')
lat_long_srs = deathCity_geo_df[1].str.replace('(','')
#strip out the ( )
lat_long_srs = lat_long_srs.str.replace(')','') 
lat_long_srs
# split the lat long series into a new data frame
lat_long_df = lat_long_srs.str.split(',',expand=True)

latitude_sr = lat_long_df[0]
longitude_sr = lat_long_df[1]
# append the lat and longs into the dataframe
clean_opioid_df["latitude"] = latitude_sr
clean_opioid_df["longitude"] = longitude_sr

clean_opioid_df.head()

C:\Users\camon\AppData\Local\Temp\ipykernel_6504\749114057.py:21: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  lat_long_srs = deathCity_geo_df[1].str.replace('(','')
C:\Users\camon\AppData\Local\Temp\ipykernel_6504\749114057.py:23: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  lat_long_srs = lat_long_srs.str.replace(')','')


,Date,DateType,Age,Sex,Race,Ethnicity,ResidenceCity,ResidenceCounty,ResidenceState,InjuryCity,...,Xylazine,Gabapentin,OpiateNOS,Heroin/Morph/Codeine,OtherOpioid,AnyOpioid,Other,DeathCityGeo,latitude,longitude
0,5/29/2012,Date of death,37.0,Male,Black,N,STAMFORD,FAIRFIELD,N,STAMFORD,...,N,N,N,N,N,N,N,"CT\n(41.575155, -72.738288)",41.575155,-72.738288
1,6/27/2012,Date of death,37.0,Male,White,N,NORWICH,NEW LONDON,N,NORWICH,...,N,N,N,N,N,N,N,"Norwich, CT\n(41.524304, -72.075821)",41.524304,-72.075821
2,3/24/2014,Date of death,28.0,Male,White,N,HEBRON,N,N,HEBRON,...,N,N,N,N,N,N,N,"Marlborough, CT\n(41.632043, -72.461309)",41.632043,-72.461309
3,12/31/2014,Date of death,26.0,Female,White,N,BALTIC,N,N,N,...,N,N,N,N,N,N,N,"Baltic, CT\n(41.617221, -72.085031)",41.617221,-72.085031
4,1/16/2016,Date of death,41.0,Male,White,N,SHELTON,FAIRFIELD,CT,SHELTON,...,N,N,N,N,N,Y,N,"Bridgeport, CT\n(41.179195, -73.189476)",41.179195,-73.189476


In [7]:
clean_opioid_df.to_csv("Resources/OD_data.csv", index=True, header=True)  #Do not keep or create the index column for loading into SQL database